# Example of using the PyRadiomics toolbox in Python

First, import some built-in Python modules needed to get our testing data.
Second, import the toolbox, only the `featureextractor` is needed, this module handles the interaction with other parts of the toolbox.

In [1]:
from __future__ import print_function
import six
import os  # needed navigate the system to get the input data

import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

### Setting up data

Test cases can be downloaded to temporary files. This is handled by the `radiomics.getTestCase()` function, which checks if the requested test case is available and if not, downloads it. It returns a tuple with the location of the image and mask of the requested test case, or (None, None) if it fails.

Alternatively, if the data is available somewhere locally, this directory can be passed as a second argument to `radiomics.getTestCase()`. If that directory does not exist or does not contain the testcase, functionality reverts to default and tries to download the test data.

If getting the test case fails, PyRadiomics will log an error explaining the cause.

In [2]:
# Get the testCase
# imagePath, maskPath = radiomics.getTestCase('brain1')
# 730_HC.png
# 596_HC.png
imagePath = '/mnt/storage/fangyijie/HC18/image/000_HC.png'
maskPath = '/mnt/storage/fangyijie/HC18/mask/000_HC_Annotation.png'
image_name = '730_HC.png'

if imagePath is None or maskPath is None:  # Something went wrong, in this case PyRadiomics will also log an error
    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

# Additonally, store the location of the example parameter file, stored in \pyradiomics\examples/exampleSettings
paramPath = os.path.join('exampleSettings', 'Params.yaml')
print('Image Path:', imagePath)
print('Mask Path:', maskPath)
print('Parameter file, absolute path:', os.path.abspath(paramPath))

Image Path: /mnt/storage/fangyijie/HC18/image/000_HC.png
Mask Path: /mnt/storage/fangyijie/HC18/mask/000_HC_Annotation.png
Parameter file, absolute path: /home/fangyijie/radiomics/exampleSettings/Params.yaml


### Instantiating the extractor

Now that we have our input, we need to define the parameters and instantiate the extractor.
For this there are three possibilities:

1. Use defaults, don't define custom settings

2. Define parameters in a dictionary, control filters and features after initialisation

3. Use a parameter file

##### Method 1, use defaults

In [3]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(force2D=True)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': True, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}


##### Method 2, hard-coded settings:

In [16]:
# First define the settings
settings = {}
settings['binWidth'] = 20
settings['sigma'] = [1, 2, 3]

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)  # ** 'unpacks' the dictionary in the function call

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default parameters

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 20, 'sigma': [1, 2, 3]}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}


In [17]:
# This cell is equivalent to the previous cell
extractor = featureextractor.RadiomicsFeatureExtractor(binWidth=20, sigma=[1, 2, 3])  # Equivalent of code above

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default parameters

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 20, 'sigma': [1, 2, 3]}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}


In [18]:
# Enable a filter (in addition to the 'Original' filter already enabled)
extractor.enableImageTypeByName('LoG')
print('')
print('Enabled filters:\n\t', extractor.enabledImagetypes)

# Disable all feature classes, save firstorder
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)

# Specify some additional features in the GLCM feature class
extractor.enableFeaturesByName(glcm=['Autocorrelation', 'Homogeneity1', 'SumSquares'])
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)


Enabled filters:
	 {'Original': {}, 'LoG': {}}

Enabled features:
	 {'firstorder': []}

Enabled features:
	 {'firstorder': [], 'glcm': ['Autocorrelation', 'Homogeneity1', 'SumSquares']}


##### Method 3, using a parameter file

In [4]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath, force2D=True, label=255)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': True, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'shape2D': None, 'firstorder': None, 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'glrlm': None, 'glszm': None, 'gldm': None}


### Extract features

image = sitk.ReadImage(ImageFilePath)Now that we have our extractor set up with the correct parameters, we can start extracting features:

In [3]:
import SimpleITK as sitk
image = sitk.ReadImage(maskPath)
image_array = sitk.GetArrayFromImage(image)

In [4]:
import numpy as np
from PIL import Image

In [5]:
np.unique(image_array)

array([  0, 255], dtype=uint8)

In [6]:
np.min(image_array)

np.uint8(0)

In [7]:
np.max(image_array)

np.uint8(255)

In [7]:
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase

In [9]:
image = sitk.ReadImage(imagePath)
mask = sitk.ReadImage(maskPath)

In [12]:
image

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x7fc1d3de27f0> >

In [10]:
image.GetDimension()

2

In [11]:
mask.GetDimension()

2

In [8]:
result = extractor.execute(imagePath, maskPath)

/home/fangyijie/miniconda3/envs/rad_user/lib/python3.11/site-packages/radiomics/imageoperations.py:125: RuntimeWarning: overflow encountered in scalar add
  highBound = maximum + 2 * binWidth


In [10]:
print('Result type:', type(result))  # result is returned in a Python ordered dictionary)
print('')
print('Calculated features')
ind = 0
for key, value in six.iteritems(result):
    print('\t', key, ':', value)
    ind += 1

print(f'there are {ind} features extracted')

Result type: <class 'collections.OrderedDict'>

Calculated features
	 diagnostics_Versions_PyRadiomics : v3.0.1
	 diagnostics_Versions_Numpy : 2.0.0
	 diagnostics_Versions_SimpleITK : 2.3.1
	 diagnostics_Versions_PyWavelet : 1.6.0
	 diagnostics_Versions_Python : 3.11.9
	 diagnostics_Configuration_Settings : {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': True, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}
	 diagnostics_Configuration_EnabledImageTypes : {'Original': {}}
	 diagnostics_Image-original_Hash : 38ba78ad9df6b9ac7b00ca2e47519b84d7628f68
	 diagnostics_Image-original_Dimensionality : 2D
	 diagnostics_Image-original_Spacing : (1.0, 1.0)
	 diagnostics_Image-original_Size : (800, 540)
	 diagnostics_Image-original_

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame([result], columns=result.keys())

In [13]:
df['image_name'] = image_name

In [14]:
df

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,image_name
0,v3.0.1,2.0.0,2.3.1,1.6.0,3.11.9,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},38ba78ad9df6b9ac7b00ca2e47519b84d7628f68,2D,"(1.0, 1.0)",...,0.0,4.0,8.934493223955046,35.737972895820185,2.2336233059887616,0.25,0.11293074377902648,0.45172297511610593,0.02823268594475662,730_HC.png


### Perform feature extraction on HC18

In [15]:
import pandas as pd

In [16]:
# Get the testCase
image_dir = '/mnt/storage/fangyijie/HC18/image'
mask_dir = '/mnt/storage/fangyijie/HC18/mask'

paramPath = os.path.join('exampleSettings', 'Params.yaml')
print('Image Folder:', image_dir)
print('Mask Folder:', mask_dir)
print('Parameter file, absolute path:', os.path.abspath(paramPath))

Image Folder: /mnt/storage/fangyijie/HC18/image
Mask Folder: /mnt/storage/fangyijie/HC18/mask
Parameter file, absolute path: /home/fangyijie/radiomics/exampleSettings/Params.yaml


In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
all_features = []
ind = 0

for img in os.listdir(image_dir):
    img_name = os.fsdecode(img)
    if img_name.endswith(".png"):
        mask_name = img_name.replace(".png", "_Annotation.png")
        img_path = os.path.join(image_dir, img_name)
        mask_path = os.path.join(mask_dir, mask_name)
        # print(img_path)
        # print(mask_path)
        # print(img_name)
        
        try:
            feature = extractor.execute(img_path, mask_path)
            df = pd.DataFrame([feature], columns=feature.keys())
            df['image_name'] = img_name
            all_features.append(df)
            # print(df)
        except Exception as error:
            print("An exception occurred:", error)
            print(f"{img_path} is not processed")

        if ind % 10 == 0:
            print(f"{ind} images are processed")
        ind += 1

0 images are processed
10 images are processed
20 images are processed
30 images are processed
40 images are processed
50 images are processed
60 images are processed
70 images are processed
80 images are processed
90 images are processed
100 images are processed
110 images are processed
120 images are processed
130 images are processed
140 images are processed
150 images are processed
160 images are processed
170 images are processed
180 images are processed
190 images are processed
200 images are processed
210 images are processed
220 images are processed
230 images are processed
240 images are processed
250 images are processed
260 images are processed
270 images are processed
280 images are processed
290 images are processed
300 images are processed
310 images are processed
320 images are processed
330 images are processed
340 images are processed
350 images are processed
360 images are processed
370 images are processed
380 images are processed
390 images are processed
400 images 

In [51]:
all_features_df = pd.concat(all_features)

In [52]:
all_features_df.shape

(997, 96)

### Save features locally

In [21]:
trg_dir = '/mnt/storage/fangyijie/HC18/radiomics'
trg_csv = 'features.csv'

In [22]:
all_features_df.to_csv(os.path.join(trg_dir, trg_csv))